<a href="https://colab.research.google.com/github/dscho15/DeepLearning/blob/main/Lecture1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lecture 1

This lecture, we implement a KNN classifier to recognize images from a known dataset: MNIST.



In [60]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

In [25]:
import torch
from torchvision import datasets

mnist_train_dataset = datasets.MNIST('', train=True, download=True);
mnist_test_dataset = datasets.MNIST('', train=False, download=True); 

# The tensor sizes of the data labels etc.

mnist_train_labels = mnist_train_dataset.targets.numpy();
mnist_train_data = mnist_train_dataset.data.numpy().reshape((-1, 28*28));

mnist_test_labels = mnist_test_dataset.targets.numpy();
mnist_test_data = mnist_test_dataset.data.numpy().reshape((-1, 28*28))

print("The train labls and data: ", mnist_train_labels.shape, mnist_train_data.shape)
print("The test labels and data: ", mnist_test_labels.shape, mnist_test_data.shape)

The train labls and data:  (60000,) (60000, 784)
The test labels and data:  (10000,) (10000, 784)


In [30]:
cifar10_train_dataset = datasets.CIFAR10('../', train=True, download=True);
cifar10_test_dataset = datasets.CIFAR10('../', train=False, download=True);

cifar10_train_labels = np.array(cifar10_train_dataset.targets)
cifar10_train_data = np.array(cifar10_train_dataset.data)

cifar10_test_labels = np.array(cifar10_test_dataset.targets)
cifar10_test_data = np.array(cifar10_test_dataset.data)

print("The train labls and data: ", cifar10_train_labels.shape, cifar10_train_data.shape)
print("The test labels and data: ", cifar10_test_labels.shape, cifar10_test_data.shape)

def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140]);

cifar10_train_data_gray = rgb2gray(cifar10_train_data).reshape((-1, 32*32))
cifar10_test_data_gray = rgb2gray(cifar10_test_data).reshape((-1, 32*32))

print("The train labls and data(gray): ", cifar10_train_labels.shape, cifar10_train_data_gray.shape)
print("The test labels and data(gray): ", cifar10_test_labels.shape, cifar10_test_data_gray.shape)


Files already downloaded and verified
Files already downloaded and verified
The train labls and data:  (50000,) (50000, 32, 32, 3)
The test labels and data:  (10000,) (10000, 32, 32, 3)
The train labls and data(gray):  (50000,) (50000, 1024)
The test labels and data(gray):  (10000,) (10000, 1024)


In [57]:
def KNN(train_data, train_label, test_data, test_label, labels, shuffle = [True, 10000], k = [1, 5, 10, 25]):

  results = []
  confusion_matrices = []

  if shuffle[0] == True:
    indices = np.random.permutation(train_data.shape[0])
    train_data = train_data[indices]
    train_label = train_label[indices]
    train_data = train_data[:shuffle[1]]
    train_label = train_label[:shuffle[1]]

  for i in k:

    # Make a classifier
    KNC = KNeighborsClassifier(i, algorithm='brute', n_jobs = -1)

    # Fit the classifiers
    KNC.fit(train_data, train_label) 

    # Get the results
    prediction = KNC.predict(test_data)

    # How well do we perform with k = 5?
    results.append(np.sum(prediction == test_label)/test_label.shape[0])

    # Get the confusion matrix
    confusion_matrices.append(confusion_matrix(test_label, prediction, labels))

  return results, confusion_matrices, k

In [62]:
mnist_results, mnist_cfmatrix, _ = KNN(mnist_train_data, mnist_train_labels, mnist_test_data, mnist_test_labels, mnist_test_dataset.classes)
cifar10_results, cifar10_cfmatrix, _ = KNN(cifar10_train_data_gray, cifar10_train_labels, cifar10_test_data_gray, cifar10_test_labels, cifar10_test_dataset.classes)

KeyboardInterrupt: ignored

In [61]:
print(mnist_results, cifar10_results, mnist_test_dataset.classes)

[0.9504, 0.9488, 0.944, 0.9304] [0.2472, 0.2545, 0.2611, 0.2562] ['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four', '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']
